In [2]:
import sys
sys.path.append("..")
from src.utilities import *

result = transcribe("../artifacts/audio/example.mp3", "medium.en")

  1%|          | 2756/320960 [00:36<1:09:31, 76.28frames/s]


KeyboardInterrupt: 

### Chapterization
Feed segments to Llama-70b to generate chapter titles.

In [3]:
import json
import sys
sys.path.append("..")
from src.utilities import *


# Segment the transcript on all pauses
# segments = chunk_on_pause(result['segments'], -1, 250, 1500)

with open('tmp.txt', 'r') as file:
    segments = json.loads(file.read())

# Create Chapter titles for segments
chapters = chapterize(segments[:5])

for c in chapters:
    print(c["title"])
    print(c["time"])
    print(c["metadata"], "\n")
    print(c["text"], "\n\n")

with open('../artifacts/data/chapters.json', 'w') as file:
    file.write(json.dumps(chapters, indent=4))

"Introduction to MIT's Course 600: Computer Science and Programming"
[00:00:00,000 --> 00:06:07,000]
(Chunk 1, 1167 words, 0:06:07.120000 mins) 

 The following content is provided under a Creative Commons license. Your support will help MIT OpenCourseWare continue to offer high quality educational resources for free. To make a donation or view additional materials from hundreds of MIT courses, visit MIT OpenCourseWare at ocw.mit.edu. PROFESSOR ERIC GRIMSON Good morning. Try it again. Good morning. Thank you. This is 600, also known as Introduction to Computer Science and Programming. My name is Eric Grimson. Together, Professor John Guttag over here. We're going to be lecturing the course this term. I want to give you a heads up. You're getting some serious firepower this term. John was department head for 10 years, felt like a century. And in course 6, I'm the current department head in course 6. John's been lecturing for 30 years, roughly. All right. I'm the young guy. I've only bee

### Note Generation
Feed the chapters back to Llama-70b to create useful notes from the transcript.

In [2]:
import json
import sys
sys.path.append("..")
from src.utilities import *

# q = "Imagine you are attending the lecture and tasked with refining the following excerpt into notes. Your goal is to streamline the content by eliminating redundancies and extraneous details. As you organize the notes, feel free to create subheadings where necessary. Your focus should be on retaining crucial information.\n\n"
q = "Write structured notes ONLY in bulletpoint form using the following excerpt of a lecture transcript:\n"

with open('../artifacts/data/chapters.json', 'r') as file:
    chapters = json.loads(file.read())

note_arr=[]
for c in chapters:
    content = '\n'.join([c['title'], c['metadata'], c['text']])
    
    notes = query(q, content)
    filtered ='\n'.join([line for line in notes.split('\n') if line.strip() and line.strip()[0] in ('*', '+', '-')])
    
    print('\n', filtered, '\n')
    
    note_arr.append({'title': c['title'], 'content': filtered})
    
with open('../artifacts/data/notes.json', 'w') as file:
    file.write(json.dumps(note_arr, indent=4))


 - The content is provided under a Creative Commons license.
- The course, 600, is also known as Introduction to Computer Science and Programming.
- Course lecturers are Professor Eric Grimson and Professor John Guttag, who have been lecturing for 25 and 30 years respectively.
- Study of computation is seen as very important.
- Course goals: Ability to understand and implement the concepts and tools of computational thinking; to think like a computer scientist.
- Strategic course goals: Preparing undergraduate students interested in Course 6, especially beginners in programming; enabling students not majoring in Course 6 to write and read small pieces of code; understanding the role computation can play in solving technical problems; building the confidence and competence for computational problem-solving.
- The course is mainly beneficial for students who are new to programming. 
- Students who have extensive programming experience may not benefit as much from the course.
- Key skill

In [11]:
# Make notes of the notes

q = "Write a <50 character summary of the notes provided. Please return ONLY the notes.\n"

sum_arr = []
for n in note_arr:
    summary = query(q, n['content'])
    print(summary)
    sum_arr.append({'title': n['title'], 'content': summary})
    
with open('../artifacts/data/summary.json', 'w') as file:
    file.write(json.dumps(sum_arr, indent=4))

Course 600: Introduction to Computer Science & Programming taught by Professors Grimson & Guttag. Content under a Creative Commons license, it teaches computational thinking, coding, and computational problem-solving. Ideal for beginners to programming.
The course taught latest skill of mapping scientific problems into computational framework. Course details include two lectures, one recitation each week, and nine hours of class work primarily involving Python programming. Course uses external libraries/code.
Understand course materials and codes. Two quizzes, dates TBA, conflicts to be communicated. Special needs noted 2 weeks prior to quiz. Quizzes are open book/notes. Problem sets 55% of grade, quizzes 45% including final. Course mainly paperless via website. Recitations on Fridays.
Course requires sign-ups for recitation, optional Python textbook, attendance not mandatory but participation encouraged, no set course text, reading is expected but not fully covered in materials, low T

In [3]:
import json
import sys
sys.path.append("..")
from src.utilities import *

# Make summary of the summary
p= "Write a 5 sentence abstract of the following notes.\n"

with open('../artifacts/data/summary.json', 'r') as file:
    sum_arr = json.loads(file.read())

txt = '\n'.join([s['content'] for s in sum_arr])

abstract = query(p, txt)

print(abstract)

with open('../artifacts/data/abstract.txt', 'w') as file:
    file.write(abstract)

MIT's Course 600: Computer Science and Programming is designed to introduce the basics of computation, aiming to prepare potential majors for advanced courses and empower non-majors with the ability to write and read small code pieces. The target audience is students with minimal to zero programming knowledge, while students with extensive programming knowledge are advised to seek alternative courses. The course encourages understanding of computational possibilities and limitations and emphasizes critical skills such as code reading and writing. The course is primarily conducted online, with additional materials and lectures posted on the website. Twice weekly lectures, a Friday recitation, and regular problem sets form the core structure of the course, while grades are derived 55% from problem sets and 45% from quizzes or final exam.


In [13]:
import json
import sys
sys.path.append("..")
from src.utilities import *

# Create a md file with everything we have

with open("../artifacts/data/abstract.txt", 'r') as f:
    abstract = f.read()

with open("../artifacts/data/summary.json", 'r') as f:
    summary = json.loads(f.read())

with open("../artifacts/data/notes.json", 'r') as f:
    notes = json.loads(f.read())
    
# Generate the markdown file
sections=[]

for s, n in zip(summary, notes):
    title = n['title']
    n_body = n['content']
    summary_body = s['content']
    
    # Format note_body into markdown
    p="Format the following notes into rich markdown.\n Indent knowledge, bold keywords, and italicize examples. Format appropriate data into tables.\n"
    
    formatted = query(p, n_body)
    
    section_md = f"## {title}\n{summary_body}\n{formatted}\n"
    
    sections.append(section_md)
    print(section_md)

md = f"""# Lecture Notes

## Abstract
{abstract}

{''.join(sections)}
"""

with open("../artifacts/data/notes.md", 'w') as f:
    f.write(md)

KeyboardInterrupt: 